In [1]:
import torch
from torch import nn

import numpy as np

In [91]:
text = ['hey how are you','good i am fine','have a nice day','my name is arpit','this is RNN class example']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

In [92]:
char2int

{'p': 0,
 'y': 1,
 'm': 2,
 'v': 3,
 'u': 4,
 'd': 5,
 't': 6,
 'h': 7,
 'w': 8,
 'R': 9,
 'i': 10,
 ' ': 11,
 'a': 12,
 'x': 13,
 'o': 14,
 'e': 15,
 'f': 16,
 'n': 17,
 's': 18,
 'l': 19,
 'r': 20,
 'c': 21,
 'N': 22,
 'g': 23}

In [93]:
''.join(text)

'hey how are yougood i am finehave a nice daymy name is arpitthis is RNN class example'

In [94]:
# Finding the length of the longest string in our data
maxlen = len(max(text, key=len))

# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of
# the sentence matches the length of the longest sentence
for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' '

In [95]:
maxlen

25

In [96]:
text

['hey how are you          ',
 'good i am fine           ',
 'have a nice day          ',
 'my name is arpit         ',
 'this is RNN class example']

In [97]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
  input_seq.append(text[i][:])
    
    # Remove first character for target sequence
  target_seq.append(text[i][:])
  print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: hey how are you          
Target Sequence: hey how are you          
Input Sequence: good i am fine           
Target Sequence: good i am fine           
Input Sequence: have a nice day          
Target Sequence: have a nice day          
Input Sequence: my name is arpit         
Target Sequence: my name is arpit         
Input Sequence: this is RNN class example
Target Sequence: this is RNN class example


In [98]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

In [99]:
input_seq

[[7,
  15,
  1,
  11,
  7,
  14,
  8,
  11,
  12,
  20,
  15,
  11,
  1,
  14,
  4,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11],
 [23,
  14,
  14,
  5,
  11,
  10,
  11,
  12,
  2,
  11,
  16,
  10,
  17,
  15,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11],
 [7,
  12,
  3,
  15,
  11,
  12,
  11,
  17,
  10,
  21,
  15,
  11,
  5,
  12,
  1,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11],
 [2,
  1,
  11,
  17,
  12,
  2,
  15,
  11,
  10,
  18,
  11,
  12,
  20,
  0,
  10,
  6,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11],
 [6,
  7,
  10,
  18,
  11,
  10,
  18,
  11,
  9,
  22,
  22,
  11,
  21,
  19,
  12,
  18,
  18,
  11,
  15,
  13,
  12,
  2,
  0,
  19,
  15]]

In [100]:
dict_size = len(char2int)
seq_len = maxlen 
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [101]:
dict_size, batch_size, seq_len

(24, 5, 25)

In [102]:
char2int


{'p': 0,
 'y': 1,
 'm': 2,
 'v': 3,
 'u': 4,
 'd': 5,
 't': 6,
 'h': 7,
 'w': 8,
 'R': 9,
 'i': 10,
 ' ': 11,
 'a': 12,
 'x': 13,
 'o': 14,
 'e': 15,
 'f': 16,
 'n': 17,
 's': 18,
 'l': 19,
 'r': 20,
 'c': 21,
 'N': 22,
 'g': 23}

[ 7, 15,  1, 11,  7, 14,  8, 11, 12, 20, 15, 11,  1, 14,  4, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 23, 14, 14,  5, 11, 10, 11, 12,  2,
       11, 16, 10, 17, 15, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  7,
       12,  3, 15, 11, 12, 11, 17, 10, 21, 15, 11,  5, 12,  1, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11,  2,  1, 11, 17, 12,  2, 15, 11, 10, 18,
       11, 12, 20,  0, 10,  6, 11, 11, 11, 11, 11, 11, 11, 11, 11,  6,  7,
       10, 18, 11, 10, 18, 11,  9, 22, 22, 11, 21, 19, 12, 18, 18, 11, 15,
       13, 12,  2,  0, 19, 15]

[ 2, 14,  0, 11,  2, 13,  3, 11, 12,  9, 14, 11,  0, 13,  5, 16, 13,
       13,  1, 11,  7, 11, 12,  8, 11, 15,  7,  4, 14, 11,  2, 12,  6, 14,
       11, 12, 11,  4,  7, 10, 14, 11,  1, 12,  0]
       
array([14,  0, 11,  2, 13,  3, 11, 12,  9, 14, 11,  0, 13,  5, 13, 13,  1,
       11,  7, 11, 12,  8, 11, 15,  7,  4, 14, 11, 14,  6, 14, 11, 12, 11,
        4,  7, 10, 14, 11,  1, 12,  0], dtype=int64)
        
[12,  0, 11,  2, 13,  3, 11, 12,  9, 14, 11,  0, 13,  5, 13, 13,  1,
       11,  7, 11, 12,  8, 11, 15,  7,  4, 14, 11, 12,  6, 14, 11, 12, 11,
        4,  7, 10, 14, 11,  1, 12,  0]

In [103]:
# Input shape --> (Batch Size, Sequence Length, One-Hot Encoding Size)
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)

In [105]:
input_seq.shape

(5, 25, 24)

In [106]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [107]:
target_seq[0]

tensor([ 7., 15.,  1., 11.,  7., 14.,  8., 11., 12., 20., 15., 11.,  1., 14.,
         4., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.])

In [108]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [109]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        print('out',out.shape, 'hidden', hidden.shape)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        print('out1', out.shape)
        out = self.fc(out)
        print('out2',out.shape)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [110]:
device = 'cpu'
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=15, n_layers=2)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)
#input_seq.to(device)
target_seq.to(device)

# Define hyperparameters
n_epochs = 1000
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [111]:
# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    input_seq.to(device)
    model.to(device)
    print('input',input_seq.shape)
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input 

out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
Epoch: 90/1000............. Loss: 0.0999
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 

input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input 

out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out to

out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 t

...

input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
Epoch: 650/1000............. Loss: 0.0017
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([1

input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input 

out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
Epoch: 810/1000............. Loss: 0.0012
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125,

out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
Epoch: 890/1000............. Loss: 0.0010
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125,

out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1, 5, 15])
out1 torch.Size([125, 15])
out2 torch.Size([125, 24])
Epoch: 980/1000............. Loss: 0.0008
input torch.Size([5, 25, 24])
out torch.Size([5, 25, 15]) hidden torch.Size([1,

In [112]:
target_seq.view(-1).long()

tensor([ 7, 15,  1, 11,  7, 14,  8, 11, 12, 20, 15, 11,  1, 14,  4, 11, 11, 11,
        11, 11, 11, 11, 11, 11, 11, 23, 14, 14,  5, 11, 10, 11, 12,  2, 11, 16,
        10, 17, 15, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  7, 12,  3, 15,
        11, 12, 11, 17, 10, 21, 15, 11,  5, 12,  1, 11, 11, 11, 11, 11, 11, 11,
        11, 11, 11,  2,  1, 11, 17, 12,  2, 15, 11, 10, 18, 11, 12, 20,  0, 10,
         6, 11, 11, 11, 11, 11, 11, 11, 11, 11,  6,  7, 10, 18, 11, 10, 18, 11,
         9, 22, 22, 11, 21, 19, 12, 18, 18, 11, 15, 13, 12,  2,  0, 19, 15])

In [113]:
np.argmax(output.detach().numpy(),axis=1)

array([ 7, 15,  1, 11,  7, 14,  8, 11, 12, 20, 15, 11,  1, 14,  4, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 23, 14, 14,  5, 11, 10, 11, 12,  2,
       11, 16, 10, 17, 15, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  7,
       12,  3, 15, 11, 12, 11, 17, 10, 21, 15, 11,  5, 12,  1, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11,  2,  1, 11, 17, 12,  2, 15, 11, 10, 18,
       11, 12, 20,  0, 10,  6, 11, 11, 11, 11, 11, 11, 11, 11, 11,  6,  7,
       10, 18, 11, 10, 18, 11,  9, 22, 22, 11, 21, 19, 12, 18, 18, 11, 15,
       13, 12,  2,  0, 19, 15], dtype=int64)

In [114]:
target_seq.shape

torch.Size([5, 25])

In [115]:
output.shape

torch.Size([125, 24])

# Torch.nonzero()

In [116]:
torch.nonzero(torch.Tensor([[1,2,3,4],[3,0,0,0]]))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:766.)
  """Entry point for launching an IPython kernel.


tensor([[0, 0],
        [0, 1],
        [0, 2],
        [0, 3],
        [1, 0]])

In [137]:
torch.Tensor([[1,2,3,4],[3,0,0,0]]) * torch.Tensor([2])

tensor([[2., 4., 6., 8.],
        [6., 0., 0., 0.]])

In [136]:
 iou_mask = (torch.Tensor([.7]) < torch.Tensor([.5])).float().unsqueeze(1)

In [133]:
iou_mask.shape

torch.Size([1, 1])